#### Code to download and separate temperature and geopotential height data.

So the previous code that I wrote for this needed to be modified to account for the fact that I will more than likely need to uh ... weight by latitude. I am a dummy and forgor to consider this. So this re-attempt at code adds in the saving of lat/lon dimensions. 
I have the relevant years and dates of temp/gph, yes, but I need to actually go in now and create the value arrays that distinguish my actual desired POR. 

In [2]:
#import statements... I think these are all of the relevant ones to what I am doing here. 
import datetime as dt
import numpy as np
import xarray as xr
import math
import netCDF4
import os
import scipy.stats
import pickle
import pandas as pd

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


I'm going to loop through gph and temp dates to create arrays of values for the levels and whatnot that I am looking for. 

In [1]:
year = [i for i in range(1997,2022)] #indicate years for data, needed for opening files
#year = [i for i in range(1999,2023)] 
#index = [i for i in range(1,43)]

In [3]:
len(year)

25

In [4]:
#create empty arrays for the years of data
gph_1000 = np.empty((len(year),152,361,1440))
gph_1000[:] = np.nan

In [ ]:
#this is my attempt at looping through gph and combining the files lol
for i in range(len(year)):
    print(year[i])
    #start by designating leap years because those arrays will be of a different size
    ##1959
    #if year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
    #if year[i] == 1959 or year[i] == 1963 or year[i] == 1967 or year[i] == 1971 or year[i] == 1975 or year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995:

    ###1979
    if year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995:
    #if year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
    #if year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
        gfile1 = xr.open_dataset("../era5/gph/era5_gph_"+str(year[i])+".nc")
        g_files1 = gfile1["z"] 
        g_data1 = g_files1.loc[dict(level=100)]
        time_coord1 = g_data1.time.values
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("../era5/gph/era5_gph_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["z"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(level=100)]
        time_coord2 = g_data2.time.values
        values2 =  g_data2.values

        #combine into np array
        gph_1000[i,:61,:,:] = values1[90:,:,:]
        gph_1000[i,61:,:,:] = values2[:91,:,:] 
        continue
        
    #this loop is because of the weird indexing in years of leap years
    ###1959
    #if year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
    #if  year[i] == 1960 or  year[i] == 1964 or  year[i] == 1968 or  year[i] == 1972 or  year[i] == 1976 or year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996:

    ###1979
    if year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996:
    #if year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
    #if year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
        gfile1 = xr.open_dataset("../era5/gph/era5_gph_"+str(year[i])+".nc")
        g_files1 = gfile1["z"] #next line will reduce this to the mean 
        g_data1 = g_files1.loc[dict(level=100)]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("../era5/gph/era5_gph_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["z"] #next line will reduce this to the mean
        g_data2 = g_files2.loc[dict(level=100)]
        values2 =  g_data2.values

        #combine into np array
        gph_1000[i,:61,:,:] = values1[91:,:,:]
        gph_1000[i,61:120,:,:] = values2[:59,:,:]
        gph_1000[i,121:,:,:] = values2[60:91,:,:]
        continue
    #everything else and repeat 
    else:
        gfile1 = xr.open_dataset("../era5/gph/era5_gph_"+str(year[i])+".nc")
        g_files1 = gfile1["z"] #next line will reduce this to the mean over th
        g_data1 = g_files1.loc[dict(level=100)]
        values1 =  g_data1.values

        gfile2 = xr.open_dataset("../era5/gph/era5_gph_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["z"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(level=100)]
        values2 =  g_data2.values
        
        gph_1000[i,:61,:,:] = values1[90:,:,:]
        gph_1000[i,61:120,:,:] = values2[:59,:,:]
        gph_1000[i,121:,:,:] = values2[60:91,:,:]
        continue

In [5]:
#this is my attempt at looping through t and combining the files lol
for i in range(len(year)):
    print(year[i])
    #start by designating leap years because those arrays will be of a different size
    ##1959
    if year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
    #if year[i] == 1959 or year[i] == 1963 or year[i] == 1967 or year[i] == 1971 or year[i] == 1975 or year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995:

    ###1979
    #if year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995:
    #if year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
    #if year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
        gfile1 = xr.open_dataset("../era5/t/era5_t_"+str(year[i])+".nc")
        g_files1 = gfile1["t"] 
        g_data1 = g_files1.loc[dict(level=100)]
        time_coord1 = g_data1.time.values
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("../era5/t/era5_t_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["t"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(level=100)]
        time_coord2 = g_data2.time.values
        values2 =  g_data2.values

        #combine into np array
        gph_1000[i,:61,:,:] = values1[90:,:,:]
        gph_1000[i,61:,:,:] = values2[:91,:,:] 
        continue
        
    #this loop is because of the weird indexing in years of leap years
    ###1959
    if year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
    #if  year[i] == 1960 or  year[i] == 1964 or  year[i] == 1968 or  year[i] == 1972 or  year[i] == 1976 or year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996:

    ###1979
    #if year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996:
    #if year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
    #if year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
        gfile1 = xr.open_dataset("../era5/t/era5_t_"+str(year[i])+".nc")
        g_files1 = gfile1["t"] #next line will reduce this to the mean 
        g_data1 = g_files1.loc[dict(level=100)]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("../era5/t/era5_t_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["t"] #next line will reduce this to the mean
        g_data2 = g_files2.loc[dict(level=100)]
        values2 =  g_data2.values

        #combine into np array
        gph_1000[i,:61,:,:] = values1[91:,:,:]
        gph_1000[i,61:120,:,:] = values2[:59,:,:]
        gph_1000[i,121:,:,:] = values2[60:91,:,:]
        continue
    #everything else and repeat 
    else:
        gfile1 = xr.open_dataset("../era5/t/era5_t_"+str(year[i])+".nc")
        g_files1 = gfile1["t"] #next line will reduce this to the mean over th
        g_data1 = g_files1.loc[dict(level=100)]
        values1 =  g_data1.values

        gfile2 = xr.open_dataset("../era5/t/era5_t_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["t"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(level=100)]
        values2 =  g_data2.values
        
        gph_1000[i,:61,:,:] = values1[90:,:,:]
        gph_1000[i,61:120,:,:] = values2[:59,:,:]
        gph_1000[i,121:,:,:] = values2[60:91,:,:]
        continue

1997


/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'rasterio' loading failed:
(numpy 1.24.3 (/nfs/home11/grad/2020/ef935217/.local/lib/python3.8/site-packages), Requirement.parse('numpy<1.23.0,>=1.16.5'), {'scipy'})
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [ ]:
##print to check
#gph_1000

In [6]:
gph_1000.shape

(25, 152, 361, 1440)

In [7]:
pickle.dump(gph_1000, open("FullNH_100_temp_1959_PT2.p", 'wb')) #download was changed because temperature
#i also did NOT divide by 10 here to conserve memory

Okay, great. This should represent an xarray of my gph data from ERA5 at 1000hPa. It's for the full, uh ... NH.

The cells below this are going to be for temperature and meridional wind. 


In [ ]:
#create empty arrays for the years of data
NH_temp_100 = np.empty((44,152,361,1440))
NH_temp_100[:] = np.nan

In [ ]:
##Full NH temp
for i in range(len(year)):
    #print(year[i])
    #start by designating leap years because those arrays will be of a different size
    if year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
        tfile1 = xr.open_dataset("../era5/t/era5_t_"+str(year[i])+".nc")
        t_files1 = tfile1["t"] 
        t_data1 = t_files1.loc[dict(level=1000)] #designate lat/lon locale
        values1 =  t_data1.values
        
        #file 2 for end
        tfile2 = xr.open_dataset("../era5/t/era5_t_"+str(year[i]+1)+".nc")
        t_files2 = tfile2["t"] 
        t_data2 = t_files2.loc[dict(level=1000)] #designate lat/lon locale
        values2 =  t_data2.values

        #combine into np array
        NH_temp_100[i,:61,:,:] = values1[90:,:,:]
        NH_temp_100[i,61:,:,:] = values2[:91,:,:]
        continue
        
    #this loop is because of the weird indexing in years of leap years
    if year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
        #file 1 for start of year
        tfile1 = xr.open_dataset("../era5/t/era5_t_"+str(year[i])+".nc")
        t_files1 = tfile1["t"] 
        t_data1 = t_files1.loc[dict(level=1000)] #designate lat/lon locale
        values1 =  t_data1.values
        
        #file 2 for end
        tfile2 = xr.open_dataset("../era5/t/era5_t_"+str(year[i]+1)+".nc")
        t_files2 = tfile2["t"] #next line will reduce this to the mean
        t_data2 = t_files2.loc[dict(level=1000)] #designate lat/lon locale
        values2 =  t_data2.values

        #combine into np array
        NH_temp_100[i,:61,:,:] = values1[91:,:,:]
        NH_temp_100[i,61:120,:,:] = values2[:59,:,:]
        NH_temp_100[i,121:,:,:] = values2[60:91,:,:]
        continue
    #everything else and repeat 
    else:
        tfile1 = xr.open_dataset("../era5/t/era5_t_"+str(year[i])+".nc")
        t_files1 = tfile1["t"] 
        t_data1 = t_files1.loc[dict(level=1000)] #designate lat/lon locale
        values1 =  t_data1.values

        tfile2 = xr.open_dataset("../era5/t/era5_t_"+str(year[i]+1)+".nc")
        t_files2 = tfile2["t"] 
        t_data2 = t_files2.loc[dict(level=1000)] #designate lat/lon locale
        values2 =  t_data2.values
        
        NH_temp_100[i,:61,:,:] = values1[90:,:,:]
        NH_temp_100[i,61:120,:,:] = values2[:59,:,:]
        NH_temp_100[i,121:,:,:] = values2[60:91,:,:]
        continue

In [ ]:
pickle.dump(NH_temp_100, open("FullNH_temp_1979.p", 'wb'))

## Imma throw in a really quick pair of cells to download data for eddy heat flux. 

In [ ]:
v100 = np.empty((44,152,361,1440))
v100[:] = np.nan

In [ ]:
for i in range(len(year)):
    print(year[i])
    #start by designating leap years because those arrays will be of a different size
    if year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
        vfile1 = xr.open_dataset("../era5/u/era5_uv_"+str(year[i])+".nc")
        v_files1 = vfile1["v"] #next line will reduce this to the mean over th
        v_data1 = v_files1.loc[dict(level=100)]
        values1 =  v_data1.values
        
        #file 2 for end
        vfile2 = xr.open_dataset("../era5/u/era5_uv_"+str(year[i]+1)+".nc")
        v_files2 = vfile2["v"] #next line will reduce this to the mean 
        v_data2 = v_files2.loc[dict(level=100)]
        values2 =  v_data2.values

        #combine into np array
        v100[i,:61] = values1[90:]
        v100[i,61:] = values2[:91]
        continue
        
    #this loop is because of the weird indexing in years of leap years
    if year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
        #file 1 for start of year
        vfile1 = xr.open_dataset("../era5/u/era5_uv_"+str(year[i])+".nc")
        v_files1 = vfile1["v"] #next line will reduce this to the mean 
        v_data1 = v_files1.loc[dict(level=100)]
        values1 =  v_data1.values
        
        #file 2 for end
        vfile2 = xr.open_dataset("../era5/u/era5_uv_"+str(year[i]+1)+".nc")
        v_files2 = vfile2["v"] #next line will reduce this to the mean
        v_data2 = v_files2.loc[dict(level=100)]
        values2 =  v_data2.values

        #combine into np array
        v100[i,:61] = values1[91:]
        v100[i,61:120] = values2[:59]
        v100[i,121:] = values2[60:91]
        continue
    #everything else and repeat 
    else:
        vfile1 = xr.open_dataset("../era5/u/era5_uv_"+str(year[i])+".nc")
        v_files1 = vfile1["v"] #next line will reduce this to the mean over th
        v_data1 = v_files1.loc[dict(level=100)]
        values1 =  v_data1.values

        vfile2 = xr.open_dataset("../era5/u/era5_uv_"+str(year[i]+1)+".nc")
        v_files2 = vfile2["v"] #next line will reduce this to the mean 
        v_data2 = v_files2.loc[dict(level=100)]
        values2 =  v_data2.values
        
        v100[i,:61] = values1[90:]
        v100[i,61:120] = values2[:59]
        v100[i,121:] = values2[60:91]
        continue

In [ ]:
v100.shape

In [ ]:
pickle.dump(v100, open("FullNH_v100_1979.p", 'wb'))

In [ ]:
t100 = np.empty((44,152,361,1440))
t100[:] = np.nan

In [ ]:
for i in range(len(year)):
    print(year[i])
    #start by designating leap years because those arrays will be of a different size
    if year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
        gfile1 = xr.open_dataset("../era5/t/era5_t_"+str(year[i])+".nc")
        g_files1 = gfile1["t"] #next line will reduce this to the mean over th
        g_data1 = g_files1.loc[dict(level=100)]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("../era5/t/era5_t_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["t"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(level=100)]
        values2 =  g_data2.values

        #combine into np array
        t100[i,:61] = values1[90:]
        t100[i,61:] = values2[:91]
        continue
        
    #this loop is because of the weird indexing in years of leap years
    if year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
        #file 1 for start of year
        gfile1 = xr.open_dataset("../era5/t/era5_t_"+str(year[i])+".nc")
        g_files1 = gfile1["t"] #next line will reduce this to the mean 
        g_data1 = g_files1.loc[dict(level=100)]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("../era5/t/era5_t_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["t"] #next line will reduce this to the mean
        g_data2 = g_files2.loc[dict(level=100)]
        values2 =  g_data2.values

        #combine into np array
        t100[i,:61] = values1[91:]
        t100[i,61:120] = values2[:59]
        t100[i,121:] = values2[60:91]
        continue
    #everything else and repeat 
    else:
        gfile1 = xr.open_dataset("../era5/t/era5_t_"+str(year[i])+".nc")
        g_files1 = gfile1["t"] #next line will reduce this to the mean over th
        g_data1 = g_files1.loc[dict(level=100)]
        values1 =  g_data1.values

        gfile2 = xr.open_dataset("../era5/t/era5_t_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["t"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(level=100)]
        values2 =  g_data2.values
        
        t100[i,:61] = values1[90:]
        t100[i,61:120] = values2[:59]
        t100[i,121:] = values2[60:91]
        continue

In [ ]:
t100.shape

In [ ]:
pickle.dump(t100, open("FullNH_t100_1979.p", 'wb'))